In [7]:
#Classify antimicrobial peptides
#Data and paper https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-3327-y

import numpy as np
import pandas as pd
import keras
from keras.preprocessing.sequence import pad_sequences
from keras_self_attention import SeqSelfAttention

In [8]:
from Bio import SeqIO
import numpy as np 

positives_train=[]
negatives_train=[]
positives_test=[]
negatives_test=[]

for seq_record in SeqIO.parse('AMP.tr.fa', "fasta"):
    
    positives_train.append(str(seq_record.seq))
    
for seq_record in SeqIO.parse('DECOY.tr.fa', "fasta"):
    
    negatives_train.append(str(seq_record.seq))
    
for seq_record in SeqIO.parse('DECOY.te.fa', "fasta"):
    
    negatives_test.append(str(seq_record.seq)) 
    
for seq_record in SeqIO.parse('AMP.te.fa', "fasta"):
    
    positives_test.append(str(seq_record.seq))
    
    
X_train=list(positives_train+negatives_train)
X_train=np.array(X_train)

y_train=list(np.ones(np.array(positives_train).shape[0]))+list(np.zeros(np.array(negatives_train).shape[0]))
y_train=np.array(y_train)

X_test=list(positives_test+negatives_test)
X_test=np.array(X_train)

y_test=list(np.ones(np.array(positives_test).shape[0]))+list(np.zeros(np.array(negatives_test).shape[0]))
y_test=np.array(y_train)
    

In [9]:
import numpy as np
import re
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^arndcqeghilkmfpstwyvx]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

# create a label encoder with alphabet
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','r','n','d','c','q','e','g','h','i','l','k','m','f','p','s','t','w','y','v','x','z']))

def ordinal_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 1.25 # A
    float_encoded[float_encoded == 1] = 2.50 # R
    float_encoded[float_encoded == 2] = 3.75 # N
    float_encoded[float_encoded == 3] = 4.30 # D
    float_encoded[float_encoded == 4] = 5.45 # C
    float_encoded[float_encoded == 5] = 6.35 # Q
    float_encoded[float_encoded == 6] = 7.85 # E 
    float_encoded[float_encoded == 7] = 8.65 # G
    float_encoded[float_encoded == 8] = 9.95 # H
    float_encoded[float_encoded == 9] = 11.25 # I
    float_encoded[float_encoded == 10] = 12.55 # L
    float_encoded[float_encoded == 11] = 13.15 # K
    float_encoded[float_encoded == 12] = 14.11 # L
    float_encoded[float_encoded == 13] = 15.29 # K
    float_encoded[float_encoded == 14] = 16.39 # M
    float_encoded[float_encoded == 15] = 17.05 # F
    float_encoded[float_encoded == 16] = 18.09 # P
    float_encoded[float_encoded == 17] = 19.49 # S
    float_encoded[float_encoded == 18] = 20.79 # T
    float_encoded[float_encoded == 19] = 21.19 # W
    float_encoded[float_encoded == 20] = 22.55 # Y
    float_encoded[float_encoded == 21] = 23.95 # V
    float_encoded[float_encoded == 22] = 24.45 # X
    float_encoded[float_encoded == 10] = 0.0 # anything else z
    

    return float_encoded

In [10]:
X=list(X_train)
for i in X_test : 
    X.append(i) 

X=[np.array(ordinal_encoder(string_to_array(i))) for i in X]
X=np.array(X)
X=pad_sequences(X)

X=X.reshape(X.shape[0],X.shape[1],1)

X_train=X[0:X_train.shape[0]]
X_test=X[X_train.shape[0]:]


In [11]:
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)

In [12]:
import keras

NUM_CLASSES = 2

# import necessary building blocks
from keras.models import Sequential
from keras.layers import Conv1D,  Flatten, Dense, Activation, Dropout,BatchNormalization,LSTM, MaxPool1D
from keras.layers.advanced_activations import LeakyReLU

In [33]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    model = Sequential()
    
    model.add(Conv1D(input_shape=X_train[0].shape,padding="same",kernel_size=3,filters=16))
    model.add(LSTM(100, return_sequences=True))
    model.add(MaxPool1D())
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())


    
    
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())


    model.add(Dropout(0.25))
    model.add(Conv1D(padding="same",kernel_size=3,filters=32))
    model.add(MaxPool1D())
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())

    model.add(Conv1D(padding="same",kernel_size=3,filters=64))
    model.add(MaxPool1D())
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())

    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(LeakyReLU(0.1))
    
    model.add(Activation("softmax"))
    
    return model



In [34]:
model = make_model()
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_48 (Conv1D)           (None, 158, 16)           64        
_________________________________________________________________
lstm_12 (LSTM)               (None, 158, 100)          46800     
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 79, 100)           0         
_________________________________________________________________
leaky_re_lu_63 (LeakyReLU)   (None, 79, 100)           0         
_________________________________________________________________
batch_normalization_48 (Batc (None, 79, 100)           400       
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 79, 32)            9632      
_________________________________________________________________
leaky_re_lu_64 (LeakyReLU)   (None, 79, 32)          

In [47]:
from keras import backend as K
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 50


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 2-way classification
    optimizer=keras.optimizers.Adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# fit model
model.fit(
    X_train, y_train2,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    
    validation_data=(X_test, y_test2),
    shuffle=True,
    verbose=1
)

Epoch 1/50
45/45 [==============================] - 9s 204ms/step - loss: 0.7478 - accuracy: 0.5084 - val_loss: 0.6927 - val_accuracy: 0.5169
Epoch 2/50
45/45 [==============================] - 8s 175ms/step - loss: 0.7084 - accuracy: 0.5253 - val_loss: 0.6924 - val_accuracy: 0.5162
Epoch 3/50
45/45 [==============================] - 8s 174ms/step - loss: 0.6925 - accuracy: 0.5162 - val_loss: 0.6945 - val_accuracy: 0.5000
Epoch 4/50
45/45 [==============================] - 8s 172ms/step - loss: 0.6819 - accuracy: 0.5702 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 5/50
45/45 [==============================] - 8s 175ms/step - loss: 0.6896 - accuracy: 0.5808 - val_loss: 0.6885 - val_accuracy: 0.5211
Epoch 6/50
45/45 [==============================] - 8s 172ms/step - loss: 0.6739 - accuracy: 0.5850 - val_loss: 0.6834 - val_accuracy: 0.5513
Epoch 7/50
45/45 [==============================] - 8s 171ms/step - loss: 0.6592 - accuracy: 0.6194 - val_loss: 0.6825 - val_accuracy: 0.5463
Epoch 

In [48]:
test_predictions = model.predict(X_test).argmax(axis=-1)

In [49]:
test_answers = y_test2.argmax(axis=-1)

In [50]:
test_accuracy = np.mean(test_predictions==test_answers)

In [51]:
print(str(test_accuracy*100)+"%")

95.2247191011236%


In [52]:
#Design new ones using an LSTM
#Using code from https://github.com/tadeaspaule/universal-name-generator

In [53]:
from tensorflow.keras.layers import LSTM, Dense, Input, concatenate, Reshape, Dropout, Bidirectional
from tensorflow.keras.models import Model, load_model

In [70]:
from Bio import SeqIO
import numpy as np 

positives_train=[]
negatives_train=[]
positives_test=[]
negatives_test=[]

for seq_record in SeqIO.parse('AMP.tr.fa', "fasta"):
    
    positives_train.append(str(seq_record.seq))
    
for seq_record in SeqIO.parse('DECOY.tr.fa', "fasta"):
    
    negatives_train.append(str(seq_record.seq))
    
for seq_record in SeqIO.parse('DECOY.te.fa', "fasta"):
    
    negatives_test.append(str(seq_record.seq)) 
    
for seq_record in SeqIO.parse('AMP.te.fa', "fasta"):
    
    positives_test.append(str(seq_record.seq))
    
    
X_train=list(positives_train+negatives_train)
X_train=np.array(X_train)

y_train=list(np.ones(np.array(positives_train).shape[0]))+list(np.zeros(np.array(negatives_train).shape[0]))
y_train=np.array(y_train)

X_test=list(positives_test+negatives_test)
X_test=np.array(X_train)



X=list(X_train)
for i in list(X_test) : 
    X.append(i) 

In [71]:
def process_names(names,*,unwanted=[]):
    names = [name for name in names]
    print("Total names:",len(names))
    chars = sorted(list(set(''.join(names))))

    def has_unwanted(word):
        for char in word:
            if char in unwanted:
                return True
        return False
    names = [name for name in names if not has_unwanted(name)]
    print("Amount of names after removing those with unwanted characters\n:",len(names))
    chars = [char for char in chars if char not in unwanted]
    print("Using the following characters:\n",chars)



    return names,chars


names,chars = process_names(X)

Total names: 2848
Amount of names after removing those with unwanted characters
: 2848
Using the following characters:
 ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']


In [72]:
def make_sequences(names,seqlen):
    sequences, lengths, nextchars = [],[],[] # To have the model learn a more macro understanding, 
                                             # it also takes the word's length so far as input
    for name in names:
        if len(name) <= seqlen:
            sequences.append(name + chars[-1]*(seqlen - len(name)))
            nextchars.append(chars[-1])
            lengths.append(len(name))
        else:
            for i in range(0,len(name)-seqlen+1):
                sequences.append(name[i:i+seqlen])
                if i+seqlen < len(name):
                    nextchars.append(name[i+seqlen])
                else:
                    nextchars.append(chars[-1])
                lengths.append(i+seqlen)

    print(len(sequences),"sequences of length",seqlen,"made")
    
    return sequences,lengths,nextchars

seqlen = 4
sequences,lengths,nextchars = make_sequences(names,seqlen)

89504 sequences of length 4 made


In [73]:
def make_onehots(*,sequences,lengths,nextchars,chars):
    x = np.zeros(shape=(len(sequences),len(sequences[0]),len(chars)), dtype='float32') # sequences
    x2 = np.zeros(shape=(len(lengths),max(lengths))) # lengths

    for i, seq in enumerate(sequences):
        for j, char in enumerate(seq):
            x[i,j,chars.index(char)] = 1.

    for i, l in enumerate(lengths):
        x2[i,l-1] = 1.

    y = np.zeros(shape=(len(nextchars),len(chars)))
    for i, char in enumerate(nextchars):
        y[i,chars.index(char)] = 1.
    
    return x,x2,y

x,x2,y = make_onehots(sequences=sequences,
                     lengths=lengths,
                     nextchars=nextchars,
                     chars=chars)

In [74]:
def get_dictchars(names,seqlen):
    dictchars = [{} for _ in range(seqlen)]

    for name in names:
        if len(name) < seqlen:
            continue
        dictchars[0][name[0]] = dictchars[0].get(name[0],0) + 1
        for i in range(1,seqlen):
            if dictchars[i].get(name[i-1],0) == 0:
                dictchars[i][name[i-1]] = {name[i]: 1}
            elif dictchars[i][name[i-1]].get(name[i],0) == 0:
                dictchars[i][name[i-1]][name[i]] = 1
            else:
                dictchars[i][name[i-1]][name[i]] += 1
    return dictchars
                
dictchars = get_dictchars(names,seqlen)
                

def generate_start_seq(dictchars):
    res = "" # The starting sequence will be stored here
    p = sum([n for n in dictchars[0].values()]) # total amount of letter occurences
    r = np.random.randint(0,p) # random number used to pick the next character
    tot = 0
    for key, item in dictchars[0].items():
        if r >= tot and r < tot + item:
            res += key
            break
        else:
            tot += item

    for i in range(1,len(dictchars)):
        ch = res[-1]
        if dictchars[i].get(ch,0) == 0:
            l = list(dictchars[i].keys())
            ch = l[np.random.randint(0,len(l))]
        p = sum([n for n in dictchars[i][ch].values()])
        r = np.random.randint(0,p)
        tot = 0
        for key, item in dictchars[i][ch].items():
            if r >= tot and r < tot + item:
                res += key
                break
            else:
                tot += item
    return res

In [75]:
def sample(preds,temperature=0.4):
    preds = np.asarray(preds).astype('float64')
    if temperature == 0:
        # Avoiding a division by 0 error
        return np.argmax(preds)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

def generate_name(model,start,*,chars=chars,temperature=0.4):
    maxlength = model.layers[3].input.shape[1]
    seqlen = int(model.layers[0].input.shape[1])
    result = start
    
    sequence_input = np.zeros(shape=(1,seqlen,len(chars)))
    for i, char in enumerate(start):
        sequence_input[0,i,chars.index(char)] = 1.
    
    length_input = np.zeros(shape=(1,maxlength))
    length_input[0,len(result)-1] = 1.
    
    prediction = model.predict(x=[sequence_input,length_input])[0]
    char_index = sample(prediction,temperature)
    while char_index < len(chars)-1 and len(result) < maxlength:
        result += chars[char_index]
        
        sequence_input = np.zeros(shape=(1,seqlen,len(chars)))
        for i, char in enumerate(result[(-seqlen):]):
            sequence_input[0,i,chars.index(char)] = 1.
        
        length_input[0,len(result)-2] = 0.
        length_input[0,len(result)-1] = 1.
        
        prediction = model.predict(x=[sequence_input,length_input])[0]
        char_index = sample(prediction,temperature)
    
    return result.title()

def generate_random_name(model,*,chars=chars,dictchars=dictchars,temperature=0.4):
    start = generate_start_seq(dictchars)
    return generate_name(model,start,chars=chars,temperature=temperature)

In [76]:
def make_model(x,x2,chars):
    inp1 = Input(shape=x.shape[1:]) # sequence input
    inp2 = Input(shape=x2.shape[1:]) # length input
    lstm = Bidirectional(LSTM(len(chars),activation='relu',dropout=0.3))(inp1)
    lstm2 = Bidirectional(LSTM(len(chars),dropout=0.3,go_backwards=True))(inp1)
    concat = concatenate([lstm,lstm2,inp2])
    dense = Dense(len(chars),activation='softmax')(concat)

    model = Model([inp1,inp2],dense)
    model.compile(optimizer='adam',loss='binary_crossentropy')
    return model

model = make_model(x,x2,chars)

In [77]:
#trying only for 3 epochs. Increase number of epochs for better results and more sequences.
generated=[]

def try_model(model,*,x=x,x2=x2,y=y,chars=chars,dictchars=dictchars,total_epochs=3,print_every=1,temperature=0.4,verbose=True):
    for i in range(total_epochs//print_every):
        history = model.fit([x,x2],y,
                            epochs=print_every,
                            batch_size=64,
                            validation_split=0.05,
                            verbose=0)

        generated.append((generate_random_name(model,chars=chars,dictchars=dictchars,temperature=temperature)).upper())
    

In [78]:
try_model(model)

In [79]:
generated

['EIMDLKLAIAGIKVCTGVKIDKEF',
 'VVSLLAKKIGKSLKGKIAGIGAAAAAAGHGGACGGAGGVHGGAG',
 'ILLCADCKLLKKLAKKIRKK']